In [ ]:
import akshare as ak
import pandas as pd

In [ ]:
ak.__version__

# 基金列表

- 约22620个

## 全部基金

In [ ]:
import akshare as ak

fund_name_em_df = ak.fund_name_em()
fund_name_em_df

## 指数基金

In [ ]:
fi = ak.fund_info_index_em(symbol="全部", indicator="全部")
fi

## 场内基金


In [ ]:
ak.fund_exchange_rank_em()

# 筛选

## 筛选条件代码

In [ ]:
def and_filter_fund(
    data_source: pd.DataFrame,
    top_number: int,
    steady_growth: bool = False,
    steady_growth_rate: float = 3,
) -> pd.DataFrame:
    # top_month_codes = set(data_source.nlargest(top_number, "近1月")["基金代码"])
    # top_3months_codes = set(data_source.nlargest(top_number, "近3月")["基金代码"])
    top_6months_codes = set(data_source.nlargest(top_number, "近6月")["基金代码"])
    top_year_codes = set(data_source.nlargest(top_number, "近1年")["基金代码"])
    top_2year_codes = set(data_source.nlargest(top_number, "近2年")["基金代码"])
    top_3year_codes = set(data_source.nlargest(top_number, "近3年")["基金代码"])

    # 计算共同基金代码的交集
    common_codes = (
        # top_month_codes &
        # top_3months_codes
        top_6months_codes
        & top_year_codes
        & top_2year_codes
        & top_3year_codes
    )

    if steady_growth:
        steady_growth_codes = set(
            data_source[
                ((data_source["近1年"] - data_source["近6月"]) > steady_growth_rate)
                & ((data_source["近2年"] - data_source["近1年"]) > steady_growth_rate)
                & ((data_source["近3年"] - data_source["近2年"]) > steady_growth_rate)
            ]["基金代码"]
        )
        common_codes = common_codes & steady_growth_codes

    # 筛选出这些共同基金
    return data_source[data_source["基金代码"].isin(common_codes)]

## 开放基金筛选

- 开放共16473个,少于所有基金6000个


### 混合型筛选


In [ ]:
bond_fund = ak.fund_open_fund_rank_em(symbol="混合型")
and_filter_fund(bond_fund, 150)

### 股票型筛选


In [ ]:
bond_fund = ak.fund_open_fund_rank_em(symbol="股票型")
and_filter_fund(bond_fund, 70)

### 债券型筛选


In [ ]:
and_filter_fund(ak.fund_open_fund_rank_em(symbol="债券型"), 500, steady_growth=True)

### 指数型筛选

In [ ]:
and_filter_fund(ak.fund_open_fund_rank_em(symbol="指数型"), 150)

## 货币基金筛选


In [ ]:
and_filter_fund(ak.fund_money_rank_em(), 30)

## 场内基金筛选


In [ ]:
and_filter_fund(ak.fund_exchange_rank_em(), 50)

## 指数基金筛选

### 债券指数

In [ ]:
and_filter_fund(
    ak.fund_info_index_em(symbol="债券指数", indicator="全部"),
    100,
    steady_growth=True,
    steady_growth_rate=4,
)

### 股票指数

In [ ]:
and_filter_fund(
    ak.fund_info_index_em(symbol="股票指数", indicator="全部"),
    200
)